<a href="https://colab.research.google.com/github/Hilpaaam/Python-AI/blob/master/Netsuite_llm_Virtual_Assistant_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Integrating the NetSuite LLM API (the `N/llm` module in SuiteScript) is a bit different from integrating a general-purpose LLM like OpenAI, because `N/llm` is primarily designed to be called *from within NetSuite* using SuiteScript.

This means that instead of your external Python backend directly calling the NetSuite LLM API, you'll likely set up a **NetSuite RESTlet or Suitelet** that acts as an intermediary. Your external Python backend will then call this NetSuite RESTlet/Suitelet, which in turn will use the `N/llm` module to interact with NetSuite's built-in generative AI capabilities.

This approach offers significant advantages for CRM:

  * **Data Stays within Oracle:** When you use `N/llm`, your data does not leave the Oracle ecosystem for model training by third parties. This is a huge privacy and security benefit for sensitive CRM data.
  * **Contextual Understanding:** The NetSuite LLM can inherently understand NetSuite data structures and terminology, potentially leading to more accurate and relevant responses for CRM tasks.
  * **Built-in Features:** You can leverage Prompt Studio and Text Enhance within NetSuite to customize the LLM's behavior and responses.

Let's break down the integration:

## Architecture for NetSuite LLM API Integration

1.  **Web Page (Frontend):** Same as before (HTML, CSS, JavaScript).
2.  **External Backend (Python/Flask):** This backend will:
      * Receive user queries from the frontend.
      * **Call your custom NetSuite RESTlet/Suitelet.**
      * Receive the AI-generated response (and potentially NetSuite data) from the RESTlet/Suitelet.
      * Send the final response back to the frontend.
3.  **NetSuite (Your Custom SuiteScript):**
      * **RESTlet (Recommended) or Suitelet:** This script will be deployed in NetSuite and act as your API endpoint.
      * It will receive the user's prompt from your external backend.
      * It will use the `N/llm` module to interact with NetSuite's generative AI.
      * It will also use other NetSuite APIs (`N/record`, `N/search`, etc.) to fetch or update CRM data based on the AI's intent.
      * It will return the processed response to your external Python backend.

## Step-by-Step Implementation

### A. NetSuite Side (SuiteScript - RESTlet)

You'll need a NetSuite developer account with SuiteScript 2.1 enabled and the `N/llm` module provisioned for your account (check Generative AI Availability in NetSuite documentation).

**1. Create a RESTlet in NetSuite:**

  * Go to **Customization \> Scripting \> Scripts \> New**.
  * Select "RESTlet" as the script type.
  * Upload your `.js` file (e.g., `netsuite_ai_restlet.js`).
  * Give it a name, ID, and deploy it to an external URL. **Crucially, enable Token-Based Authentication (TBA) and assign it to a role with appropriate permissions** (e.g., "Web Services" permission, permissions to access Customer, Sales Order, Task records, etc., and permission to use Generative AI features).
  * Note down the external URL for the RESTlet and the Consumer Key, Consumer Secret, Token ID, and Token Secret for TBA.

**`netsuite_ai_restlet.js` (NetSuite SuiteScript 2.1 RESTlet):**

```javascript
/**
 * @NApiVersion 2.1
 * @NScriptType Restlet
 * @NModuleScope Public
 */
define(['N/log', 'N/llm', 'N/record', 'N/search'],
    function (log, llm, record, search) {

        function post(requestBody) {
            var userQuery = requestBody.query;
            log.debug('Received User Query', userQuery);

            if (!userQuery) {
                return {
                    success: false,
                    message: 'No query provided.'
                };
            }

            try {
                // --- Step 1: Use N/llm to understand intent and potentially extract entities ---
                // You can send the raw user query to the LLM to understand intent
                // or use predefined prompts from Prompt Studio if you've set them up.

                // Example: Basic text generation based on the query
                // More advanced would be to use llm.evaluatePrompt if you have structured prompts.
                var llmResponse = llm.generateText({
                    prompt: "As an Oracle NetSuite CRM assistant, analyze this request: '" + userQuery + "'. " +
                            "If it's a direct question about customer, sales, or task data, indicate what data is needed and for whom. " +
                            "If it's a request to create something, identify what to create and its key details. " +
                            "If you can't fulfill it with NetSuite data, state that. " +
                            "For example: 'Get customer contact info for John Doe.', 'Create a task to follow up with ABC Corp.', 'Summarize recent sales.'",
                    modelFamily: llm.ModelFamily.COHERE_COMMAND_R_PLUS // Or LLAMA_3_1, etc.
                    // modelParameters: { maxTokens: 150, temperature: 0.7 } // Adjust as needed
                });

                var aiIntentAnalysis = llmResponse.text;
                log.debug('LLM Intent Analysis', aiIntentAnalysis);

                var finalResponse = "NetSuite AI Assistant: ";

                // --- Step 2: Implement Logic based on AI Intent Analysis (Crucial Part) ---
                // This is simplified. In a real scenario, you'd use more robust NLP/regex
                // or even a separate 'function calling' mechanism within your LLM prompt
                // to extract structured data (e.g., customer name, task description, dates).

                if (aiIntentAnalysis.toLowerCase().includes('customer contact info for')) {
                    var customerNameMatch = /customer contact info for (.*?)[\.]?/.exec(aiIntentAnalysis);
                    if (customerNameMatch && customerNameMatch[1]) {
                        var customerName = customerNameMatch[1].trim();
                        finalResponse += getCustomerContactInfo(customerName);
                    } else {
                        finalResponse += "Please specify the customer's name for contact info.";
                    }
                } else if (aiIntentAnalysis.toLowerCase().includes('create a task to')) {
                     var taskDescMatch = /create a task to (.*?)[\.]?/.exec(aiIntentAnalysis);
                     if (taskDescMatch && taskDescMatch[1]) {
                         var taskDescription = taskDescMatch[1].trim();
                         finalResponse += createTask(taskDescription);
                     } else {
                         finalResponse += "What task would you like to create?";
                     }
                }
                // ... add more conditions for other CRM actions (sales orders, opportunities, cases, etc.)
                else if (userQuery.toLowerCase().includes('hello') || userQuery.toLowerCase().includes('hi')) {
                    finalResponse += "Hello! How can I assist you with NetSuite CRM today?";
                }
                else {
                    // Fallback: If no specific action is detected, just provide the LLM's direct analysis
                    finalResponse += "I understood: '" + aiIntentAnalysis + "'. How can I proceed? (e.g., 'Can you get customer info for ABC Corp?' or 'Create a task to call John Doe').";
                }

                return {
                    success: true,
                    message: finalResponse
                };

            } catch (e) {
                log.error({
                    title: 'LLM API Error or Processing Error',
                    details: e.toString() + ' Stack: ' + e.stack
                });
                return {
                    success: false,
                    message: 'NetSuite AI Assistant encountered an internal error. Please try again or check NetSuite logs.',
                    error: e.message
                };
            }
        }

        // --- Helper functions to interact with NetSuite data ---

        function getCustomerContactInfo(customerName) {
            var customerInfo = 'No customer found with that name.';
            try {
                var customerSearch = search.create({
                    type: search.Type.CUSTOMER,
                    filters: [['companyname', search.Operator.IS, customerName]],
                    columns: [
                        'entityid', 'phone', 'email', 'altphone', 'fax',
                        search.createColumn({ name: 'address1', join: 'address' }),
                        search.createColumn({ name: 'city', join: 'address' }),
                        search.createColumn({ name: 'state', join: 'address' }),
                        search.createColumn({ name: 'zipcode', join: 'address' })
                    ]
                });

                var resultSet = customerSearch.run().getRange({ start: 0, end: 1 });

                if (resultSet && resultSet.length > 0) {
                    var customerResult = resultSet[0];
                    customerInfo = 'Found customer: ' + customerResult.getValue('entityid') + '. ';
                    customerInfo += 'Phone: ' + (customerResult.getValue('phone') || 'N/A') + '. ';
                    customerInfo += 'Email: ' + (customerResult.getValue('email') || 'N/A') + '. ';
                    var address = customerResult.getValue({ name: 'address1', join: 'address' });
                    var city = customerResult.getValue({ name: 'city', join: 'address' });
                    var state = customerResult.getValue({ name: 'state', join: 'address' });
                    var zip = customerResult.getValue({ name: 'zipcode', join: 'address' });
                    if (address || city || state || zip) {
                         customerInfo += 'Address: ' + [address, city, state, zip].filter(Boolean).join(', ');
                    }
                } else {
                    customerInfo = 'Customer "' + customerName + '" not found in NetSuite.';
                }
            } catch (e) {
                log.error({
                    title: 'Error in getCustomerContactInfo',
                    details: e.toString()
                });
                customerInfo = 'Error retrieving customer info.';
            }
            return customerInfo;
        }

        function createTask(taskDescription) {
            try {
                var newTask = record.create({
                    type: record.Type.TASK,
                    isDynamic: true
                });
                newTask.setValue({ fieldId: 'title', value: taskDescription });
                newTask.setValue({ fieldId: 'assigned', value: search.lookupFields({
                    type: search.Type.EMPLOYEE, id: 'CURRENT_USER', columns: ['entityid']
                }).entityid }); // Assign to current user running the script
                // Optionally set due date, status, etc.
                // newTask.setValue({ fieldId: 'duedate', value: new Date() }); // Example: Today
                var newTaskId = newTask.save();
                return 'Task "' + taskDescription + '" created successfully with ID: ' + newTaskId + '.';
            } catch (e) {
                log.error({
                    title: 'Error in createTask',
                    details: e.toString()
                });
                return 'Failed to create task: ' + e.message;
            }
        }


        return {
            post: post
        };
    });
```

**Important Notes for the NetSuite RESTlet:**

  * **Authentication:** Token-Based Authentication (TBA) is highly recommended for external integrations.
  * **Permissions:** The role assigned to your integration must have:
      * "REST Web Services" permission.
      * Permissions for the specific record types it interacts with (e.g., "Customers" - View/Edit, "Tasks" - Create/View/Edit, "Sales Orders" - View/Edit, "Cases" - View/Edit).
      * Permissions to access Generative AI features (check your NetSuite settings for this).
  * **Error Handling:** Include robust `try...catch` blocks.
  * **Logging:** Use `N/log` to debug your RESTlet. You can view logs in **Customization \> Scripting \> Script Deployments \> (Your RESTlet Deployment) \> Execution Log**.
  * **Asynchronous Operations:** For very long-running LLM calls or complex data processing, consider using `promise` methods from `N/llm` or `N/task` modules.
  * **`modelFamily`:** You can specify the LLM family (e.g., `llm.ModelFamily.COHERE_COMMAND_R_PLUS`, `llm.ModelFamily.LLAMA_3_1`). Check NetSuite documentation for available models in your region.
  * **Prompt Studio:** If you're using Prompt Studio in NetSuite to create pre-defined prompts, you'd use `llm.evaluatePrompt({ promptId: 'your_prompt_id', variables: {...} })` instead of `llm.generateText`. This gives you more control over the LLM's behavior from within NetSuite.

### B. External Backend (Python/Flask)

This will be similar to the OpenAI integration, but instead of calling `client.chat.completions.create`, you'll make an authenticated HTTP POST request to your NetSuite RESTlet.

**`app.py` (Flask Backend with NetSuite RESTlet Call):**

In [ ]:
import os
from flask import Flask, request, jsonify, render_template
from dotenv import load_dotenv
import requests
import json
from requests_oauthlib import OAuth1Session # For Token-Based Authentication

# Load environment variables from .env file
load_dotenv()

app = Flask(__name__, static_folder='static', template_folder='static')

# NetSuite API Configuration (from .env)
NETSUITE_RESTLET_URL = os.getenv('NETSUITE_RESTLET_URL')
NETSUITE_CONSUMER_KEY = os.getenv('NETSUITE_CONSUMER_KEY')
NETSUITE_CONSUMER_SECRET = os.getenv('NETSUITE_CONSUMER_SECRET')
NETSUITE_TOKEN_ID = os.getenv('NETSUITE_TOKEN_ID')
NETSUITE_TOKEN_SECRET = os.getenv('NETSUITE_TOKEN_SECRET')
NETSUITE_ACCOUNT_ID = os.getenv('NETSUITE_ACCOUNT_ID') # e.g., 'TSTDRV123456_SB1' for sandbox

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/api/chat', methods=['POST'])
def chat():
    user_message = request.json.get('message')
    if not user_message:
        return jsonify({"error": "No message provided"}), 400

    if not all([NETSUITE_RESTLET_URL, NETSUITE_CONSUMER_KEY, NETSUITE_CONSUMER_SECRET,
                NETSUITE_TOKEN_ID, NETSUITE_TOKEN_SECRET, NETSUITE_ACCOUNT_ID]):
        return jsonify({"error": "NetSuite API credentials not fully configured."}), 500

    try:
        # Prepare OAuth1Session for NetSuite TBA
        # Realm should be your NetSuite Account ID, often with underscores for sandboxes
        oauth = OAuth1Session(
            client_key=NETSUITE_CONSUMER_KEY,
            client_secret=NETSUITE_CONSUMER_SECRET,
            resource_owner_key=NETSUITE_TOKEN_ID,
            resource_owner_secret=NETSUITE_TOKEN_SECRET,
            realm=NETSUITE_ACCOUNT_ID # Format: 1234567_SB1 for sandbox, 1234567 for production
        )

        headers = {
            'Content-Type': 'application/json'
        }
        payload = {
            'query': user_message
        }

        # Make the POST request to your NetSuite RESTlet
        response = oauth.post(NETSUITE_RESTLET_URL, headers=headers, data=json.dumps(payload))

        if response.status_code == 200:
            netsuite_data = response.json()
            if netsuite_data.get('success'):
                ai_reply = netsuite_data.get('message', "NetSuite AI processed your request.")
            else:
                ai_reply = f"NetSuite AI encountered an issue: {netsuite_data.get('message', 'Unknown error')}"
                if netsuite_data.get('error'): # Include more details if available
                     ai_reply += f" (Details: {netsuite_data['error']})"
            return jsonify({"reply": ai_reply})
        else:
            print(f"NetSuite RESTlet Error: Status {response.status_code}, Response: {response.text}")
            return jsonify({"error": f"NetSuite RESTlet call failed. Status: {response.status_code}. Details: {response.text}"}), 500

    except Exception as e:
        print(f"Error communicating with NetSuite RESTlet: {e}")
        return jsonify({"error": "Failed to communicate with NetSuite AI. Please try again."}), 500

if __name__ == '__main__':
    app.run(debug=True, port=5000)

**`.env` file (update with your NetSuite details):**

```
# NetSuite API Details
NETSUITE_RESTLET_URL="https://[YOUR_ACCOUNT_ID].restlets.api.netsuite.com/app/site/restlet.nl?script=[SCRIPT_ID]&deploy=[DEPLOY_ID]"
NETSUITE_CONSUMER_KEY="[YOUR_CONSUMER_KEY]"
NETSUITE_CONSUMER_SECRET="[YOUR_CONSUMER_SECRET]"
NETSUITE_TOKEN_ID="[YOUR_TOKEN_ID]"
NETSUITE_TOKEN_SECRET="[YOUR_TOKEN_SECRET]"
NETSUITE_ACCOUNT_ID="[YOUR_ACCOUNT_ID_WITH_UNDERSCORE_FOR_SANDBOX_E.G._1234567_SB1]"
```

**To get your NetSuite RESTlet URL:**

1.  After deploying your RESTlet, go to **Customization \> Scripting \> Script Deployments**.
2.  Find your RESTlet deployment.
3.  The "External URL" column will show the URL. Copy this directly.
4.  The `script` and `deploy` parameters are part of the URL.

### C. Frontend (`script.js`)

No changes needed here from the previous OpenAI example, as the frontend simply calls your `/api/chat` endpoint, and your backend handles whether it talks to OpenAI or NetSuite.

## Running the Integrated System:

1.  **NetSuite Configuration:**
      * Deploy the `netsuite_ai_restlet.js` as a RESTlet in your NetSuite account.
      * Set up Token-Based Authentication for the RESTlet and a role with the necessary permissions.
      * Copy all the required credentials (URL, Consumer Key/Secret, Token ID/Secret, Account ID).
2.  **Update `.env`:** Paste your NetSuite credentials into the `.env` file in your Python project.
3.  **Install Python Libraries:** `pip install Flask requests_oauthlib python-dotenv`
4.  **Run Python Backend:** `python app.py`
5.  **Access Frontend:** Open your browser to `http://127.0.0.1:5000/`.

Now, when you type a query into the chat, your Python backend will forward it to your NetSuite RESTlet, which will then use the `N/llm` module and other NetSuite APIs to process the request and return a response back to your web page.

This is a robust and secure way to leverage NetSuite's native generative AI capabilities for your CRM virtual assistant\!

In [1]:
!python app.py

python3: can't open file '/content/app.py': [Errno 2] No such file or directory


In [2]:
%%writefile app.py
import os
from flask import Flask, request, jsonify, render_template
from dotenv import load_dotenv
import requests
import json
from requests_oauthlib import OAuth1Session # For Token-Based Authentication

# Load environment variables from .env file
load_dotenv()

app = Flask(__name__, static_folder='static', template_folder='static')

# NetSuite API Configuration (from .env)
NETSUITE_RESTLET_URL = os.getenv('NETSUITE_RESTLET_URL')
NETSUITE_CONSUMER_KEY = os.getenv('NETSUITE_CONSUMER_KEY')
NETSUITE_CONSUMER_SECRET = os.getenv('NETSUITE_CONSUMER_SECRET')
NETSUITE_TOKEN_ID = os.getenv('NETSUITE_TOKEN_ID')
NETSUITE_TOKEN_SECRET = os.getenv('NETSUSET_TOKEN_SECRET')
NETSUITE_ACCOUNT_ID = os.getenv('NETSUITE_ACCOUNT_ID') # e.g., 'TSTDRV123456_SB1' for sandbox

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/api/chat', methods=['POST'])
def chat():
    user_message = request.json.get('message')
    if not user_message:
        return jsonify({"error": "No message provided"}), 400

    if not all([NETSUITE_RESTLET_URL, NETSUITE_CONSUMER_KEY, NETSUITE_CONSUMER_SECRET,
                NETSUITE_TOKEN_ID, NETSUITE_TOKEN_SECRET, NETSUITE_ACCOUNT_ID]):
        return jsonify({"error": "NetSuite API credentials not fully configured."}), 500

    try:
        # Prepare OAuth1Session for NetSuite TBA
        # Realm should be your NetSuite Account ID, often with underscores for sandboxes
        oauth = OAuth1Session(
            client_key=NETSUITE_CONSUMER_KEY,
            client_secret=NETSUITE_CONSUMER_SECRET,
            resource_owner_key=NETSUITE_TOKEN_ID,
            resource_owner_secret=NETSUITE_TOKEN_SECRET,
            realm=NETSUITE_ACCOUNT_ID # Format: 1234567_SB1 for sandbox, 1234567 for production
        )

        headers = {
            'Content-Type': 'application/json'
        }
        payload = {
            'query': user_message
        }

        # Make the POST request to your NetSuite RESTlet
        response = oauth.post(NETSUITE_RESTLET_URL, headers=headers, data=json.dumps(payload))

        if response.status_code == 200:
            netsuite_data = response.json()
            if netsuite_data.get('success'):
                ai_reply = netsuite_data.get('message', "NetSuite AI processed your request.")
            else:
                ai_reply = f"NetSuite AI encountered an issue: {netsuite_data.get('message', 'Unknown error')}"
                if netsuite_data.get('error'): # Include more details if available
                     ai_reply += f" (Details: {netsuite_data['error']})"
            return jsonify({"reply": ai_reply})
        else:
            print(f"NetSuite RESTlet Error: Status {response.status_code}, Response: {response.text}")
            return jsonify({"error": f"NetSuite RESTlet call failed. Status: {response.status_code}. Details: {response.text}"}), 500

    except Exception as e:
        print(f"Error communicating with NetSuite RESTlet: {e}")
        return jsonify({"error": "Failed to communicate with NetSuite AI. Please try again."}), 500

if __name__ == '__main__':
    app.run(debug=True, port=5000)

Writing app.py


In [3]:
!python app.py

Traceback (most recent call last):
  File "/content/app.py", line 3, in <module>
    from dotenv import load_dotenv
ModuleNotFoundError: No module named 'dotenv'


In [4]:
!pip install Flask requests_oauthlib python-dotenv

In [5]:
!python app.py

 * Serving Flask app 'app'
 * Debug mode: on
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 777-545-785


In [ ]:
!python app.py

 * Serving Flask app 'app'
 * Debug mode: on
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 777-545-785
